In [11]:
from torch import optim
from torch.optim import Adam
from tqdm import tqdm

from utils.data import read_domain_ids_per_chain_from_txt
from utils.dataset import *
from diffusion_model.embed import *
from diffusion_model.structure_diffusion_model import *

In [4]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
EPOCH = 100
LEARNING_RATE = 1e-3

In [5]:
train_pdbs, train_pdb_chains = read_domain_ids_per_chain_from_txt('./data/train_domains.txt')
test_pdbs, test_pdb_chains = read_domain_ids_per_chain_from_txt('./data/test_domains.txt')

train_loader = BackboneCoordsDataLoader(train_pdb_chains, "./data/train_backbone_coords_20.npy", "./data/train_data_res_20.npy",seq_length=20, batch_size=128, shuffle=True)
test_loader = BackboneCoordsDataLoader(test_pdb_chains, './data/test_backbone_coords_20.npy', './data/test_data_res_20.npy', seq_length=20, batch_size=128, shuffle=True)

In [7]:
diffusion = ProteinDiffusion(timesteps = 1000, beta_schedule = 'cosine', device=DEVICE)
model = StructureModel()
optimizer = optim.Adam(params=model.parameters(), lr = 1e-3)

In [14]:

model.train()
train_loss = 0

for batch_idx, (pdb_chains, res_label, atom_coords) in enumerate(train_loader):
    # protein sequence embedding
    single_repr = get_single_representation(pdb_chains, res_label)
    print(single_repr.shape)

Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm1b_t33_650M_UR50S.pt" to /Users/jasonwong/.cache/torch/hub/checkpoints/esm1b_t33_650M_UR50S.pt
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/regression/esm1b_t33_650M_UR50S-contact-regression.pt" to /Users/jasonwong/.cache/torch/hub/checkpoints/esm1b_t33_650M_UR50S-contact-regression.pt


torch.Size([128, 1280])


KeyboardInterrupt: 